In [2]:
# ml.m5.4xlarge
!pip install ImageBind/.
!pip install git+https://github.com/huggingface/transformers.git
!pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade
!pip install accelerate==0.20.3

Processing ./ImageBind
  Preparing metadata (setup.py) ... done
  Using cached pytorchvideo-0.1.5-py3-none-any.whl
  Using cached torch-1.13.0-cp39-cp39-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchvision-0.14.0-cp39-cp39-manylinux1_x86_64.whl (24.3 MB)
  Using cached torchaudio-0.13.0-cp39-cp39-manylinux1_x86_64.whl (4.2 MB)
  Using cached timm-0.6.7-py3-none-any.whl (509 kB)
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached regex-2023.8.8-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached einops-0.6.1-py3-none-any.whl (42 kB)
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached eva_decord-0.6.1-py3-none-manylinux2010_x86_64.whl (13.6 MB)
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached types_regex-2023.8.8.0-py3-none-any.whl (5.1 kB)
  Using cached mayavi-4.8.1-cp39-cp39-linux_x86_64.whl
  Using cached Cartopy-0.22.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached n

In [3]:
from ImageBind.imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import json

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
s3 = boto3.resource('s3')
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-demo-2.2/text_repr.json"

# loading in raw data
content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T
print("length of dataframe: "+ str(len(df)))
df.head(5)

length of dataframe: 210


,arrival,eddischarge,admission,discharge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",The ED disposition was admitted at 2125-03-19 ...,The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",The ED disposition was admitted at 2154-02-05 ...,The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,NaN,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",The ED disposition was home at 2154-08-03 22:2...,The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",The ED disposition was admitted at 2153-02-13 ...,The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",The ED disposition was admitted at 2137-06-12 ...,The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...


In [5]:
df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df['medrecon'] = df['medrecon'].fillna("The patient was previously not taking any medications.")
df['pyxis'] = df['pyxis'].fillna("The patient did not receive any medications.")
df['vitals'] = df['vitals'].fillna("The patient had no vitals recorded")
df['codes'] = df['codes'].fillna("The patient received no diagnostic codes")
df = df.drop("admission",axis=1)
df = df.drop("discharge",axis=1)
# df['general'] = df['arrival'] + df['triage'] + df['codes'] + df['pyxis'] 
# df = df.drop('arrival',axis=1)
# df = df.drop('triage',axis=1)
# df = df.drop('codes',axis=1)
# df = df.drop('pyxis',axis=1)
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df

,arrival,triage,medrecon,vitals,pyxis,codes,eddischarge
37887480,"Patient 10014729, a 21 year old white - other ...","At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34176810,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient did not receive any medications.,The patient received the following diagnostic ...,1
32103106,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...,0
38797992,"Patient 10020640, a 91 year old white female, ...","At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...,1
33473053,"Patient 10015272, a 78 year old white female, ...","At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...,1
...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously not taking any medi...,The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...,1
31628990,"Patient 10009049, a 56 year old white male, ar...","At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...,1
32405286,"Patient 10004457, a 65 year old white male, ar...","At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34391979,"Patient 10004720, a 61 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...,1


In [6]:
print(df.medrecon.str.len().mean())
print(df.vitals.str.len().mean())
print(df.arrival.str.len().mean())
print(df.triage.str.len().mean())
print(df.pyxis.str.len().mean())
print(df.codes.str.len().mean())

834.6428571428571
982.6333333333333
181.45714285714286
244.43333333333334
278.247619047619
180.56190476190477


In [7]:
medrecon_list = df.medrecon.to_list()
vitals_list = df.vitals.to_list()
arrival_list = df.arrival.to_list()
triage_list = df.triage.to_list()
pyxis_list = df.pyxis.to_list()
codes_list = df.codes.to_list()

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

print("Done Loading Model")

100%|██████████| 4.47G/4.47G [00:14<00:00, 340MB/s]


Done Loading Model


In [8]:
with torch.no_grad():
    inputs_medrecon = {ModalityType.TEXT: data.load_and_transform_text(medrecon_list, device)}
    embeddings_medrecon = model(inputs_medrecon)
    inputs_vitals = {ModalityType.TEXT: data.load_and_transform_text(vitals_list, device)}
    embeddings_vitals = model(inputs_vitals)
    inputs_arrival = {ModalityType.TEXT: data.load_and_transform_text(arrival_list, device)}
    embeddings_arrival = model(inputs_general)
    inputs_triage = {ModalityType.TEXT: data.load_and_transform_text(triage_list, device)}
    embeddings_triage = model(inputs_general)
    inputs_pyxis = {ModalityType.TEXT: data.load_and_transform_text(pyxis_list, device)}
    embeddings_pyxis = model(inputs_general)
    inputs_codes = {ModalityType.TEXT: data.load_and_transform_text(codes_list, device)}
    embeddings_codes = model(inputs_general)
    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-8-59e6b344347f>:2 in <module>                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/imagebind/data.py:108 in load_and_transform_text          │
│                                                                                                  │
│   105 def load_and_transform_text(text, device):                                                 │
│   106 │   if text is None:                                                                       │
│   107 │   │   return None                                                                        │
│ ❱ 108 │   tokenizer = SimpleTokenizer(bpe_path=BPE_PATH)                                         │
│   109 │   tokens = [tokenizer(t).unsqueeze(0).to(device) for t in text]                          │
│   110 │   tokens = torch.cat(tokens, dim=0)                                                      │
│   111 │   return tokens                                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/imagebind/models/multimodal_preprocessors.py:502 in       │
│ __init__                                                                                         │
│                                                                                                  │
│   499 │   │   self.byte_encoder = bytes_to_unicode()                                             │
│   500 │   │   self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}                   │
│   501 │   │                                                                                      │
│ ❱ 502 │   │   with g_pathmgr.open(bpe_path, "rb") as fh:                                         │
│   503 │   │   │   bpe_bytes = io.BytesIO(fh.read())                                              │
│   504 │   │   │   merges: List[str] = gzip.open(bpe_bytes).read().decode("utf-8").split("\n")    │
│   505 │   │   merges = merges[1 : 49152 - 256 - 2 + 1]                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/iopath/common/file_io.py:1062 in open                     │
│                                                                                                  │
│   1059 │   │   # pass enable mode to handler that will be logging                                │
│   1060 │   │   # read, write operations separately.                                              │
│   1061 │   │   handler.set_logging(self._enable_logging)                                         │
│ ❱ 1062 │   │   bret = handler._open(path, mode, buffering=buffering, **kwargs)  # type: ignore   │
│   1063 │   │                                                                                     │
│   1064 │   │   kvs = self.__get_open_keys(path, mode, buffering)                                 │
│   1065 │   │   self.__log_tmetry_keys(handler, kvs)                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/iopath/common/file_io.py:645 in _open                     │
│                                                                                                  │
│    642 │   │   │   file: a file-like object.                                                     │
│    643 │   │   """                                                                               │
│    644 │   │   self._check_kwargs(kwargs)                                                        │
│ ❱  645 │   │   return open(  # type: ignore                

In [ ]:
with torch.no_grad():
    embeddings_medrecon = model(inputs_medrecon)
    embeddings_vitals = model(inputs_vitals)
    embeddings_arrival = model(inputs_arrival)
    embeddings_triage = model(inputs_triage)
    embeddings_pyxis = model(inputs_pyxis)
    embeddings_codes = model(inputs_codes)

In [ ]:
f = torch.eq(embeddings_medrecon['text'], embeddings_arrival['text'])
f.unique(return_counts=True)

In [ ]:
f = torch.eq(embeddings_vitals['text'], embeddings_triage['text'])
f.unique(return_counts=True)

In [ ]:
f = torch.eq(embeddings_codes['text'], embeddings_medrecon['text'])
f.unique(return_counts=True)

In [ ]:
embeddings_codes

In [ ]:
print(embeddings_arrival['text'].size())
print(embeddings_vitals['text'].size())
print(embeddings_medrecon['text'].size())
print(embeddings_codes['text'].size())
print(embeddings_pyxis['text'].size())
print(embeddings_triage['text'].size())

In [ ]:
# concatenate vectors into one
concat = torch.cat((embeddings_vitals['text'], embeddings_arrival['text'], embeddings_medrecon['text'], embeddings_codes['text'],embeddings_pyxis['text'], embeddings_triage['text']), -1)
print(concat.size())
concat[0]

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        return weighted

In [ ]:
self_attention = SelfAttention( 6144 )
output = self_attention(concat.unsqueeze(0))
output = output[0]

In [ ]:
# apply a dense layer to project them on different space.
dense_layer = nn.Linear(6144, 768)
new_vec = dense_layer(concat)
print(new_vec.size())

# TODO: What do I do with the embeddings.

# Need a way to update gradients, and stuff

In [ ]:
# Turn our tensor and match it with the label. After doing so, we feed it through a training loop to identify.

import torch

def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Output:
# torch.Size([95, 1, 1]) torch.Size([95, 1, 1])
# torch.Size([47, 1, 1]) torch.Size([47, 1, 1])
